<p style="text-align: center; font-size: 30px;"><b>Text Generation</b></p>

# Long Short-Term Memory Networks

Long short-term memory networks (LSTMs) are a specialized type of recurrent neural network (RNN) designed to capture long-range dependencies in sequential data. They are widely used in natural language processing (NLP) for tasks such as machine translation, speech generation, summarization, and dialogue systems.

## Information Persistence and Neural Networks

Human language relies on persistent structures: grammar rules, shared vocabulary, and the ability to interpret each word in context. As you read, earlier sentences influence your understanding of later ones — demonstrating long-term information retention.

Neural networks, inspired by the brain, initially struggled with such temporal relationships. Early models like perceptrons performed static transformations without memory, limiting their effectiveness on tasks requiring sequential reasoning.

## Deep Neural Networks

Modern hardware enabled deeper architectures such as:

* **MLPs (Multi-Layer Perceptrons):** Stacks of perceptrons, effective for simple input–output transformations.
* **CNNs (Convolutional Neural Networks):** Specialized for images, using filters that scan across spatial dimensions.
* **RNNs (Recurrent Neural Networks):** Designed for sequences, where each output depends on previous inputs.

RNNs “unroll” across time, producing hidden states that propagate contextual information forward.

## The Long-Term Dependency Problem

Standard RNNs struggle with long sequences due to vanishing and exploding gradients. They perform well when the relevant context is close to the prediction target (e.g., completing “The grass is always ___” → “greener”).

However, in sentences requiring distant context —
*“It is winter… The grass is always ___.”* → “brown” —
standard RNNs fail to preserve the necessary information across many timesteps.

This challenge is known as the **long-term dependency problem**.

## Long Short-Term Memory (LSTM) Networks

LSTMs were introduced to address these limitations. While maintaining the chain structure of RNNs, LSTMs replace the simple recurrent unit with a more sophisticated cell composed of multiple interacting components.

An LSTM cell processes input using **four internal operations** controlled by gates:

1. **Forget gate:** Determines how much of the previous cell state to keep.
2. **Input gate:** Regulates how much new information to write to the cell.
3. **Candidate state:** Proposes new content to add.
4. **Output gate:** Controls how much of the updated cell state forms the new hidden state.

## Cell State and Hidden State

The LSTM maintains two vector states:

* **Cell state** $c_t$: acts as long-term memory, carrying information across many timesteps.
* **Hidden state** $h_t$: short-term output used for predictions and passed to subsequent layers.

The core update equations (simplified) are:

$$
f_t = \sigma(W_f[x_t, h_{t-1}] + b_f)
$$

$$
i_t = \sigma(W_i[x_t, h_{t-1}] + b_i)
$$

$$
\tilde{c}*t = \tanh(W_c[x_t, h*{t-1}] + b_c)
$$

$$
c_t = f_t \odot c_{t-1} + i_t \odot \tilde{c}_t
$$

$$
o_t = \sigma(W_o[x_t, h_{t-1}] + b_o)
$$

$$
h_t = o_t \odot \tanh(c_t)
$$

These gating mechanisms allow LSTMs to preserve important information and discard irrelevant details as sequences evolve.

## Why LSTMs Matter

By blending persistent memory with selective updates, LSTMs excel at:

* Capturing long-term patterns
* Modeling contextual relationships over extended text
* Generating coherent sequences
* Handling variable-length inputs

They remain foundational components in many deep learning systems, even as newer models (like Transformers) gain prominence.

LSTMs provide a robust solution to the limitations of traditional RNNs, enabling machines to leverage distant context much like humans do when interpreting language.


# Introduction to seq2seq

Sequence-to-sequence (seq2seq) models extend the power of LSTMs by organizing them into a larger **encoder–decoder** architecture capable of generating entire sequences of text. While LSTMs handle temporal dependencies within a sequence, seq2seq models specialize in transforming one sequence into another — often of different length and structure.

## Where seq2seq Models Are Used

Seq2seq architectures power many modern NLP systems, including:

* Machine translation (e.g., Google Translate)
* Automatic text summarization
* Chatbots and conversational agents
* Named Entity Recognition (NER)
* Speech recognition and transcription

These applications require understanding an input sequence and producing a coherent, structured output sequence.

## Encoder–Decoder Structure

A seq2seq network is composed of two recurrent components:

### **1. Encoder**

* Accepts the input sequence (text, audio, or video).
* Processes each element step by step through an RNN (often an LSTM).
* Discards the encoder’s final output matrix but **retains its final hidden state**.
* This hidden state becomes a compressed representation — a fixed-length vector capturing the meaning of the entire input sequence.

### **2. Decoder**

* Starts with the encoder’s final state as its initial state.
* Generates tokens one at a time to form the output sequence.
* During training, uses **teacher forcing**, where the correct previous token is supplied to help the model learn the next-token prediction more efficiently.

Teacher forcing trains the decoder to predict the next character or word in the target output sequence, given all previous correct tokens.

## Why seq2seq Matters

Seq2seq models brought coherence and fluency to early neural text-generation systems by enabling networks to read an entire input, compress it into memory, and then reconstruct or transform it into a new sequence. They laid the groundwork for modern architectures such as attention-based seq2seq models and the Transformer.


# Preprocessing for seq2seq

Before building a sequence-to-sequence (seq2seq) model with TensorFlow and Keras, the first major step is **preprocessing the text data**. This ensures that both the encoder (English) and decoder (Spanish) can operate on clean, tokenized sequences with appropriate vocabulary and structure.

## Using TensorFlow and Keras

Keras is accessed directly from TensorFlow:

```python
from tensorflow import keras
```

Keras provides high-level components for model building, making it well-suited for seq2seq architectures.

## What We Need to Prepare

To train even a small English-to-Spanish translator, preprocessing must produce several key components:

### **1. Vocabulary Sets**

For both languages, we need:

* A set of all unique input tokens for English
* A set of all unique target tokens for Spanish

These vocabularies determine the dimensionality of the encoder and decoder input layers.

### **2. Token Counts**

For each language:

* The total number of unique tokens
* Used to define embedding sizes, one-hot encodings, or output layer dimensions

### **3. Maximum Sentence Lengths**

Models require fixed-length input sequences.
We compute:

* Maximum input sentence length (English)
* Maximum target sentence length (Spanish)

These values control padding and ensure uniform tensor shapes.

## Marking Start and End of Target Sequences

Seq2seq decoders must know:

* When to **start** generating text
* When to **stop** generating text

To achieve this, each target sentence is wrapped with special tokens:

* Prepend `<START>`
* Append `<END>`

Example:

```
Estoy feliz.
→ <START> Estoy feliz. <END>
```

This teaches the decoder to begin generation at `<START>` and stop once it predicts `<END>`.

## Noise Removal Considerations

Depending on your goal, you may normalize:

* Casing
* Punctuation
* Special characters

For many applications, these details are not critical, and simplifying the text can improve model performance and reduce vocabulary size.

## Before Continuing

You should examine the provided `script.py` and read each line closely. Understanding how text is loaded, tokenized, padded, and indexed will help you modify or extend the model later—for example, improving translation quality or adapting the system to new languages.

This preprocessing pipeline sets the foundation for building the encoder–decoder architecture that powers seq2seq translation.


In [8]:
# Step 1: Import required libraries and load the span-eng.txt file
from tensorflow import keras
import re

data_path = "utils/span-eng.txt"

with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
print(lines)

["We'll see.\tDespués veremos.", "We'll see.\tYa veremos.", "We'll try.\tLo intentaremos.", "We've won!\t¡Hemos ganado!", 'Well done.\tBien hecho.', "What's up?\t¿Qué hay?", 'Who cares?\t¿A quién le importa?', 'Who drove?\t¿Quién condujo?', 'Who drove?\t¿Quién conducía?', 'Who is he?\t¿Quién es él?', 'Who is it?\t¿Quién es?']


**What the code does:**

Opens the translation file in read mode, using UTF-8 encoding so that accented characters like á, é, ñ, ¿, ¡ load correctly. The file handle is stored in f. It creates a list of strings which are split at the character "\n", which represents the new line,

In [9]:
# Step 2: Initialize empty lists for sentences and vocab sets
input_docs = []
target_docs = []
input_tokens = set()
target_tokens = set()

In [10]:
# Step 3: Process each line, looping ove the lines list
for line in lines:
    # Example: line 1 = "We'll see.\tDespués veremos."
    input_doc, target_doc = line.split('\t')
    '''
    What the code does:
    Splits the translation pair into:
    - input_doc → the English sentence (before the tab, represented by '\t', which is none other than a space in the text)
    - target_doc → the Spanish sentence (after the tab)
    This is the fundamental step that separates the two languages for encoder/decoder processing.
    '''
    # Example input_doc: "We'll see."
    input_docs.append(input_doc)
    # Appends "We'll see."toinput_docs
    
    # Example target_doc: "Después veremos."
    # Split punctuation: "Después veremos ." for example
    target_doc = " ".join(re.findall(r"[\w']+|[^\s\w]", target_doc))
    '''
    What the code does:
    Uses a regular expression to separate punctuation from words, ensuring every token (words and punctuation marks) becomes an individual element when later split.
    This helps the model treat the period "." as its own token instead of attaching it to the last word.
    '''
    # Add start/end tokens:
    # "<START> Después veremos . <END>"
    target_doc = "<START> " + target_doc + " <END>"
    target_docs.append(target_doc)

    # Example tokens from input_doc: ["We", "'", "ll", "see", "."]
    for token in re.findall(r"[\w']+|[^\s\w]", input_doc):
        input_tokens.add(token)

    # Example tokens from target_doc: ["<START>", "Después", "veremos", ".", "<END>"]
    for token in target_doc.split():
        target_tokens.add(token)
        '''
        What the code does:
        Loops through every token (word or punctuation mark) in the English sentence.
        The regex ensures punctuation like "." becomes its own token, which helps build a clean, consistent vocabulary for the encoder.
        '''

In [11]:
# Step 4: Sort vocabularies
input_tokens = sorted(list(input_tokens))
target_tokens = sorted(list(target_tokens))

# Step 5: Count vocabulary sizes
num_encoder_tokens = len(input_tokens)
num_decoder_tokens = len(target_tokens)

# Step 6: Compute max sequence lengths
try:
    max_encoder_seq_length = max(
        len(re.findall(r"[\w']+|[^\s\w]", doc)) for doc in input_docs
    )
    max_decoder_seq_length = max(
        len(re.findall(r"[\w']+|[^\s\w]", doc)) for doc in target_docs
    )
except ValueError:
    max_encoder_seq_length = 0
    max_decoder_seq_length = 0

In [12]:
print(input_docs)
print(target_docs)
print(input_tokens)
print(target_tokens)

["We'll see.", "We'll see.", "We'll try.", "We've won!", 'Well done.', "What's up?", 'Who cares?', 'Who drove?', 'Who drove?', 'Who is he?', 'Who is it?']
['<START> Después veremos . <END>', '<START> Ya veremos . <END>', '<START> Lo intentaremos . <END>', '<START> ¡ Hemos ganado ! <END>', '<START> Bien hecho . <END>', '<START> ¿ Qué hay ? <END>', '<START> ¿ A quién le importa ? <END>', '<START> ¿ Quién condujo ? <END>', '<START> ¿ Quién conducía ? <END>', '<START> ¿ Quién es él ? <END>', '<START> ¿ Quién es ? <END>']
['!', '.', '?', "We'll", "We've", 'Well', "What's", 'Who', 'cares', 'done', 'drove', 'he', 'is', 'it', 'see', 'try', 'up', 'won']
['!', '.', '<END>', '<START>', '?', 'A', 'Bien', 'Después', 'Hemos', 'Lo', 'Quién', 'Qué', 'Ya', 'conducía', 'condujo', 'es', 'ganado', 'hay', 'hecho', 'importa', 'intentaremos', 'le', 'quién', 'veremos', '¡', '¿', 'él']


In [13]:
print(num_encoder_tokens) #> same as lenght input_tokens
print(num_decoder_tokens) #> same as lenght taeget_tokens
print(max_encoder_seq_length)
print(max_decoder_seq_length)

18
27
4
12


## Why the Encoder Gives Length 4 but the Decoder Gives Length 12

### (And exactly *what question* the regex “asks” to split `<START>` into 3 tokens)

The key difference comes from how the regex:

```
[\w']+|[^\s\w]
```

**interprets characters**. Let’s break that down clearly.

---

## 1. Encoder sentences → clean words + punctuation

Example encoder sentence:

```
Who is he?
```

The regex asks:

### **A) “Is this one or more word characters or apostrophes?”**

→ `[\w']+`
Matches:

* `Who`
* `is`
* `he`

### **B) “If not, is it a single character that is NOT whitespace and NOT a word character?”**

→ `[^\s\w]`
Matches:

* `?`

So the regex produces:

```
["Who", "is", "he", "?"]   → length = 4
```

This is why:

```
max_encoder_seq_length = 4
```

---

## 2. Decoder sentences → include `<START>` and `<END>` which the regex **cannot** treat as single tokens

Take this decoder sentence:

```
<START> ¿ A quién le importa ? <END>
```

Now the regex checks each character and asks the same questions:

---

## ❗ How the regex splits `<START>` into 3 tokens

### **Character-by-character interrogation:**

1. `"<"`

   * Is it `[\w']+`? → **No** (it’s not a word or apostrophe)
   * Is it `[^\s\w]` (not whitespace, not a word)? → **Yes**
     → Token: `"<"`

2. `"START"`

   * Are these word characters? → **Yes**
     → Token: `"START"`

3. `">"`

   * Is it a word? → **No**
   * Is it non-space & non-word? → **Yes**
     → Token: `">"`

### So the regex produces:

```
"<START>" → ["<", "START", ">"]   # 3 tokens
```

Same logic applies to `<END>`:

```
"<END>" → ["<", "END", ">"]
```

---

## ❗ How the rest of the Spanish sentence is processed

`¿ A quién le importa ?` becomes:

```
"¿"      → punctuation token
"A"      → word token
"quién"  → word token
"le"     → word token
"importa"→ word token
"?"      → punctuation token
```

---

## Final token list the regex produces for the decoder’s longest sentence

```
[
  '<', 'START', '>',          # <START>
  '¿', 'A', 'quién', 'le', 'importa', '?',   # Spanish content
  '<', 'END', '>'             # <END>
]
```

Count them:

```
3 (START) + 6 (Spanish) + 3 (END) = 12 tokens
```

So:

```
max_decoder_seq_length = 12
```

---

## 🔥 Final Summary

* The encoder has no `<...>` tokens → clean tokenization → max length is **4**.
* The decoder includes `<START>` and `<END>` → regex splits them into **"<"**, **"START"**, **">"**.
* The regex “asks” two questions for each match attempt:

  1. **Is this one or more word characters/apostrophes?**
  2. **If not, is it a single non-space, non-word character?**
* Because `<` and `>` are non-word characters, and “START” is a word, the regex produces **three tokens** per marker.

That is the entire reason why the decoder sequence length jumps to **12**.


# Training Setup (part 1)

To train a seq2seq model, each sentence must be transformed into a numerical format that Keras and TensorFlow can understand. This is done using **one-hot vectors** within 3-dimensional matrices representing all training samples.

## What Is a One-Hot Vector?

A one-hot vector represents a single token using zeros everywhere except the index of that token.
Given the vocabulary:

```
["the", "dog", "licked", "me"]
```

The one-hot vector for **“dog”** is:

```
[0, 1, 0, 0]
```

Each token in the sentence gets its own one-hot vector.

## Why We Need Feature Dictionaries

Before vectorizing the data, we must map each token to a numerical index.
We create four dictionaries:

* **English features dictionary:** maps English tokens → indices
* **Spanish features dictionary:** maps Spanish tokens → indices
* **Reverse English dictionary:** maps indices → English tokens
* **Reverse Spanish dictionary:** maps indices → Spanish tokens

These mappings allow us to convert words into vectors, and later convert model predictions back into words.

## Building the Encoder Input Matrix

We initialize the encoder input matrix using NumPy:

```python
import numpy as np

encoder_input_data = np.zeros(
    (len(input_docs), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
```

## Breaking Down the Encoder Matrix

The `encoder_input_data` matrix is filled with zeros initially. Its dimensions are:

1. **Number of documents**
   Each English sentence is one training example.

2. **Maximum encoder sequence length**
   All English sentences are padded or shortened to match this length.

3. **Number of encoder tokens**
   The size of the English vocabulary, determining the length of each one-hot vector.

## Why Use Float32?

Setting `dtype='float32'` ensures efficient computation on modern hardware, especially GPUs. Although one-hot vectors contain only 0s and 1s, float32 allows smooth integration with TensorFlow operations.

This setup prepares the encoder data for the process of converting tokens into one-hot vectors that feed the LSTM.


In [7]:
# Step 1: Import required libraries and preprocessed data
from tensorflow import keras
import numpy as np

# Step 2: Inspect basic dataset statistics (number of samples, tokens, and max lengths)
print('Number of samples:', len(input_docs))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

# Step 3: Build the input features dictionary (token -> index) for English
# Example: if input_tokens = ["We'll", "see", "."]
# input_features_dict might be {"We'll": 0, "see": 1, ".": 2}
input_features_dict = dict(
    (token, i) for i, token in enumerate(input_tokens)
)

# Step 4: Build the target features dictionary (token -> index) for Spanish
# Example: if target_tokens = ["<START>", "Después", "veremos", ".", "<END>"]
# target_features_dict might be {"<START>": 0, "Después": 1, "veremos": 2, ".": 3, "<END>": 4}
target_features_dict = dict(
    (token, i) for i, token in enumerate(target_tokens)
)

# Step 5: Build the reverse input features dictionary (index -> token) for English
# This lets us go from model predictions (indices) back to words.
reverse_input_features_dict = dict(
    (i, token) for token, i in input_features_dict.items()
)

# Step 6: Build the reverse target features dictionary (index -> token) for Spanish
# Same idea as above, but for the decoder language.
reverse_target_features_dict = dict(
    (i, token) for token, i in target_features_dict.items()
)

# Step 7: Create the encoder input matrix of zeros (one-hot containers for English)
# Shape: (num_samples, max_encoder_seq_length, num_encoder_tokens)
encoder_input_data = np.zeros(
    (len(input_docs), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32'
)

# Step 8: Create the decoder input matrix of zeros (one-hot containers for Spanish decoder input)
# Shape: (num_samples, max_decoder_seq_length, num_decoder_tokens)
decoder_input_data = np.zeros(
    (len(input_docs), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32'
)

# Step 9: Create the decoder target matrix of zeros (expected Spanish outputs)
# Shape: (num_samples, max_decoder_seq_length, num_decoder_tokens)
# For now, decoder_input_data and decoder_target_data will be identical shells.
decoder_target_data = np.zeros(
    (len(input_docs), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32'
)

# Step 10: Print a quick sanity check for the encoder input matrix
print(
    "\nHere's the first item in the encoder input matrix:\n",
    encoder_input_data[0],
    "\n\nThe number of columns should match the number of unique input tokens "
    "and the number of rows should match the maximum sequence length for input sentences."
)

# Step 11: (Placeholder comments for later) Build out decoder_input_data and decoder_target_data values
# In the next step, you will fill encoder_input_data, decoder_input_data,
# and decoder_target_data with actual one-hot encodings based on the
# feature dictionaries and the tokenized sentences.

Number of samples: 11
Number of unique input tokens: 18
Number of unique output tokens: 27
Max sequence length for inputs: 4
Max sequence length for outputs: 12

Here's the first item in the encoder input matrix:
 [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]] 

The number of columns should match the number of unique input tokens and the number of rows should match the maximum sequence length for input sentences.


## Key Insights: How the Three Seq2Seq Matrices Work and How They Are Built

### 1. **`encoder_input_data`** — *What the encoder reads*

* **Shape:**
  `(#samples, max_encoder_seq_length, num_encoder_tokens)`
* **Represents:**
  The **English input sentences**, converted into one-hot vectors.
* **How it is filled:**
  For each English sentence, each token position becomes a one-hot vector over the **encoder vocabulary**.
* **Purpose:**
  Feeds the encoder with the sequence it must **encode into a final hidden state**.

---

### 2. **`decoder_input_data`** — *What is fed into the decoder during training*

* **Shape:**
  `(#samples, max_decoder_seq_length, num_decoder_tokens)`
* **Represents:**
  The **Spanish target sentences**, but shifted so that each sequence **begins with `<START>`**.
* **How it is filled:**
  For each Spanish sentence, the model sees:
  `<START> token1 token2 token3 ...`
* **Purpose:**
  This is the input that the decoder consumes at each time step to learn how to produce the next token.
  *(This is part of teacher forcing.)*

---

### 3. **`decoder_target_data`** — *What the decoder is expected to output*

* **Shape:**
  `(#samples, max_decoder_seq_length, num_decoder_tokens)`
* **Represents:**
  The **expected output** for each decoder step, ending with `<END>`.
* **How it is filled:**
  For each Spanish sentence, this sequence is **shifted by one position**:
  `token1 token2 token3 ... <END>`
* **Purpose:**
  At every training step:
  `decoder_input_data[t] → decoder predicts → decoder_target_data[t]`

---

### 🔑 Why `decoder_input_data` and `decoder_target_data` start identical

They are **both zero-filled shells at initialization**, but they will be filled differently:

| Matrix                | Starts With      | Used For                               |
| --------------------- | ---------------- | -------------------------------------- |
| `decoder_input_data`  | `<START>`        | What we feed into the decoder          |
| `decoder_target_data` | first real token | What the decoder must learn to predict |

This mismatch (shift) is what trains the decoder to generate the next word.


# Training Setup (Part 2)

Before training our seq2seq model, we must convert each token in each sentence into a **one-hot vector**. These vectors are grouped into a 3-dimensional NumPy matrix that stores:

* the **sentence index**
* the **timestep** within the sentence
* the **token index** inside the vocabulary

## One-Hot Example

For the sentence:

```
["the", "dog", "licked", "me"]
```

with vocabulary `["the", "dog", "licked", "me"]`, its one-hot encoding becomes:

```
[
  [1, 0, 0, 0],  # timestep 0 → "the"
  [0, 1, 0, 0],  # timestep 1 → "dog"
  [0, 0, 1, 0],  # timestep 2 → "licked"
  [0, 0, 0, 1],  # timestep 3 → "me"
]
```

Each **row** corresponds to a timestep, and each **column** corresponds to a position in the vocabulary.

---

## Building the 3-D Training Matrices

We fill these matrices by setting:

```
matrix_name[line, timestep, features_dict[token]] = 1
```

This gives us three NumPy arrays:

### 1. **Encoder Input Data**

A sequence of one-hot vectors representing the **English input** sentence at each timestep.

### 2. **Decoder Input Data**

A sequence of one-hot vectors representing the **Spanish target sentence**, but shifted to begin with `<START>`.

### 3. **Decoder Target Data**

A sequence of one-hot vectors representing the **expected next token** in the Spanish sentence (ending with `<END>`).

---

## Why Two Decoder Matrices?

This comes from **teacher forcing**, a technique used in seq2seq training.

At timestep *t*:

* the decoder receives a Spanish token from **decoder_input_data[t]**
  (usually the correct previous token),
* and must predict the next Spanish token in **decoder_target_data[t]**.

Using the ground-truth previous token at each step makes training more stable and helps the model learn correct token transitions more quickly.

Teacher forcing, therefore, requires **two** decoder matrices:

* **decoder_input_data**: what we feed *into* the decoder
* **decoder_target_data**: what the decoder must learn to output


In [14]:
# Step 1: Loop over each English–Spanish sentence pair
for line, (input_doc, target_doc) in enumerate(zip(input_docs, target_docs)):

    # ============================================================
    # ENCODER: BUILD ONE-HOT VECTORS FOR ENGLISH INPUT SENTENCES
    # ============================================================
    # We tokenize the English sentence using the same regex used
    # for computing max_encoder_seq_length.
    # Each token is placed at encoder_input_data[line][timestep][token_index] = 1.
    # ------------------------------------------------------------
    for timestep, token in enumerate(re.findall(r"[\w']+|[^\s\w]", input_doc)):
        
        # Debug print
        print("Encoder input timestep & token:", timestep, token)
        print(input_features_dict[token])
        
        # Step 1A: Assign a 1 at the correct (line, timestep, token) position.
        # This creates a one-hot vector for the English token at this timestep.
        encoder_input_data[line, timestep, input_features_dict[token]] = 1.


    # ============================================================
    # DECODER: BUILD ONE-HOT VECTORS FOR SPANISH INPUT SENTENCES
    # ============================================================
    # Here we use target_doc.split() which is the same method used
    # when computing max_decoder_seq_length.
    #
    # decoder_input_data receives the Spanish sequence starting with <START>.
    # Each timestep contains a one-hot vector for the token.
    # ------------------------------------------------------------
    for timestep, token in enumerate(target_doc.split()):
        
        # Debug print
        print("Decoder input timestep & token:", timestep, token)
        print(target_features_dict[token])

        # Step 2: Assign 1 for decoder_input_data at this timestep.
        # This feeds <START> token first, then the Spanish tokens.
        decoder_input_data[line, timestep, target_features_dict[token]] = 1.


        # ============================================================
        # DECODER TARGET: SHIFTED ONE-HOT VECTORS FOR SPANISH OUTPUT
        # ============================================================
        # Teacher forcing:
        #   decoder_input_data[t]  → decoder predicts → decoder_target_data[t]
        #
        # That means decoder_target_data must be ahead by one timestep.
        #
        # decoder_target_data[line][timestep - 1] corresponds to the
        # expected NEXT token.
        # ------------------------------------------------------------
        if timestep > 0:

            # Debug print
            print("Decoder target timestep:", timestep)

            # Step 3: Assign the correct one-hot target at timestep - 1.
            decoder_target_data[line, timestep - 1, target_features_dict[token]] = 1.

Encoder input timestep & token: 0 We'll
3
Encoder input timestep & token: 1 see
14
Encoder input timestep & token: 2 .
1
Decoder input timestep & token: 0 <START>
3
Decoder input timestep & token: 1 Después
7
Decoder target timestep: 1
Decoder input timestep & token: 2 veremos
23
Decoder target timestep: 2
Decoder input timestep & token: 3 .
1
Decoder target timestep: 3
Decoder input timestep & token: 4 <END>
2
Decoder target timestep: 4
Encoder input timestep & token: 0 We'll
3
Encoder input timestep & token: 1 see
14
Encoder input timestep & token: 2 .
1
Decoder input timestep & token: 0 <START>
3
Decoder input timestep & token: 1 Ya
12
Decoder target timestep: 1
Decoder input timestep & token: 2 veremos
23
Decoder target timestep: 2
Decoder input timestep & token: 3 .
1
Decoder target timestep: 3
Decoder input timestep & token: 4 <END>
2
Decoder target timestep: 4
Encoder input timestep & token: 0 We'll
3
Encoder input timestep & token: 1 try
15
Encoder input timestep & token: 2 .
1

## 1. Does `decoder_target_data` get ones at timestep 0?

Look at this part:

```python
for timestep, token in enumerate(target_doc.split()):
    decoder_input_data[line, timestep, target_features_dict[token]] = 1.

    if timestep > 0:
        decoder_target_data[line, timestep-1, target_features_dict[token]] = 1.
```

* At `timestep = 0` (token = `<START>`):

  * `decoder_input_data[line, 0, idx("<START>")] = 1.` ✅
  * `if timestep > 0:` → **False**, so **no write** to `decoder_target_data`.

So **nothing is written** to `decoder_target_data` at timestep 0.
The *first* time we write to `decoder_target_data` is when `timestep = 1`.

Concretely, for:

```text
<START> Después veremos . <END>
```

We get:

| timestep | token     | write to `decoder_input_data`                     | write to `decoder_target_data`                     |
| -------- | --------- | ------------------------------------------------- | -------------------------------------------------- |
| 0        | `<START>` | `decoder_input_data[line, 0, idx("<START>")] = 1` | *(no write: timestep > 0 is False)*                |
| 1        | `Después` | `decoder_input_data[line, 1, idx("Después")] = 1` | `decoder_target_data[line, 0, idx("Después")] = 1` |
| 2        | `veremos` | `decoder_input_data[line, 2, idx("veremos")] = 1` | `decoder_target_data[line, 1, idx("veremos")] = 1` |
| 3        | `.`       | `decoder_input_data[line, 3, idx(".")] = 1`       | `decoder_target_data[line, 2, idx(".")] = 1`       |
| 4        | `<END>`   | `decoder_input_data[line, 4, idx("<END>")] = 1`   | `decoder_target_data[line, 3, idx("<END>")] = 1`   |

So:

* `decoder_input_data` has ones from timestep `0` to `4`.
* `decoder_target_data` has ones from timestep `0` to `3`.

Timestep `4` in `decoder_target_data` stays all zeros.

## 2. So yes: the last row of `decoder_target_data` is all zeros

Exactly:

* Sequence length (for this sentence) = 5 timesteps.
* We only ever write to `decoder_target_data` up to `timestep - 1`, where `timestep` goes from `1` to `4`.
* That means **the last timestep in `decoder_target_data` (index 4) is never written to** and remains all zeros.

This is expected and normal for this simple implementation.

Intuition:

* At training time, we teach the model:

  * Given `<START>` → predict `Después`
  * Given `Después` → predict `veremos`
  * Given `veremos` → predict `.`
  * Given `.` → predict `<END>`
* After `<END>`, we don’t care about “next token”, so that “next step” in `decoder_target_data` is just zeros (no target).

In a more advanced setup you might:

* apply a **mask** to ignore that last timestep in the loss, or
* pad sequences and mask padded positions.

In this Codecademy-style tutorial, they keep it simple and just leave that last row as zeros, which is ok — the model still trains fine.

## 3. Summary in one sentence

* `decoder_input_data` has tokens from `<START>` to `<END>`.
* `decoder_target_data` is the **same sentence shifted one step left**, so it **does not include the `<START>` position**, and the last timestep is unused (all zeros).


# Encoder Training Setup

To build the encoder portion of our seq2seq model, we assemble it using **Keras layers**. The encoder’s job is to read an input sentence (represented as a sequence of one-hot vectors) and compress it into two internal states that summarize the entire meaning of the sentence. These two states will then be used to initialize the decoder so it can begin generating the translated output.

This process involves defining an input layer, connecting it to an LSTM layer, and extracting the LSTM’s internal states.

---

## **1. Importing the Required Layers**

We begin by importing the Keras components that allow us to construct the encoder:

```python
from keras.layers import Input, LSTM
from keras.models import Model
```

* `Input` lets us define the shape of the data we will feed into the model.
* `LSTM` provides the recurrent layer that processes sequences step-by-step.
* `Model` will later allow us to package the encoder into a full Keras model.

---

## **2. Defining the Encoder Input Layer**

The encoder expects a 3-dimensional tensor:

1. **batch size** (number of sentences processed at once — left as `None`)
2. **timesteps** (number of tokens in a sentence — also variable → `None`)
3. **features** (the one-hot vector size = number of encoder tokens)

We specify this with:

```python
encoder_inputs = Input(shape=(None, num_encoder_tokens))
```

This tells Keras:
“Each sentence is a sequence of unknown length, and each timestep is a vector with `num_encoder_tokens` features.”

The model can now handle sentences of different lengths during training.

---

## **3. Adding the Encoder LSTM Layer**

We now create the LSTM that will process each one-hot vector one timestep at a time. We choose a hidden dimensionality of 100 — meaning the encoder will compress the entire sentence into a **100-dimensional representation**.

```python
encoder_lstm = LSTM(100, return_state=True)
```

* `100` → size of the internal memory
* `return_state=True` → instructs the LSTM to give us not only its output, but also its final hidden state (`h`) and cell state (`c`)

These states carry forward the meaning of everything the encoder has read.

---

## **4. Connecting Inputs to the LSTM**

We now pass the one-hot input sequence through the LSTM:

```python
encoder_outputs, state_hidden, state_cell = encoder_lstm(encoder_inputs)
```

This call returns:

* `encoder_outputs` → the LSTM output at every timestep (not needed here)
* `state_hidden` → final hidden state (**h**), summarizing short-term context
* `state_cell` → final cell state (**c**), summarizing long-term context

Together, these two vectors form the **sentence embedding** that the decoder will use to begin generating the translation.

---

## **5. Saving the Encoder States**

We keep only the two internal states and discard the per-timestep outputs:

```python
encoder_states = [state_hidden, state_cell]
```

These states are the *only* output of the encoder that matters. They act like a compressed memory of the entire input sentence, capturing the key structure and meaning that will guide the decoder as it produces the translated sentence.

---

The encoder is now fully constructed. It accepts a sequence of English tokens, processes it through an LSTM, and outputs a pair of internal states representing the entire sequence — exactly what the decoder needs to begin generating a translation.


In [17]:
from tensorflow import keras
from keras.layers import Input, LSTM
from keras.models import Model

# Step 1: Define the encoder input layer
# - Batch size: None (flexible)
# - Timesteps: None (variable sequence length)
# - Features: num_encoder_tokens (one-hot vector size)
encoder_inputs = Input(shape=(None, num_encoder_tokens))

# Step 2: Build the encoder LSTM layer
# - Dimensionality: 256 units
# - return_state=True so we can extract the hidden + cell states
encoder_lstm = LSTM(256, return_state=True)

# Step 3: Pass the inputs through the LSTM
# This returns:
#   encoder_outputs → full sequence outputs (unused here)
#   state_hidden     → final hidden state (h)
#   state_cell       → final cell state (c)
encoder_outputs, state_hidden, state_cell = encoder_lstm(encoder_inputs)

# Step 4: Store the two states in a list for the decoder
encoder_states = [state_hidden, state_cell]


2025-12-03 20:30:50.048517: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


# Decoder Training Setup

The decoder is the second half of the seq2seq architecture. While the encoder **reads** the input sentence and compresses it into internal states, the decoder **uses those states** to generate the output sentence step by step. Structurally, the decoder looks very similar to the encoder: it has its own input layer, its own LSTM, and a final Dense layer with a Softmax activation to produce token probabilities.

---

## 1. Decoder Inputs: Feeding the Target Language to the Model

Just as the encoder receives one-hot vectors for the **source language**, the decoder receives one-hot vectors for the **target language** (e.g., Spanish). We allow variable-length sentences using `None` for the timestep dimension:

```python
decoder_inputs = Input(shape=(None, num_decoder_tokens))
```

Here:

* `None` → variable number of timesteps (sentence length)
* `num_decoder_tokens` → size of the target vocabulary (the depth of each one-hot vector)

At each timestep, `decoder_inputs` will contain one target token (often starting with `<START>`) in one-hot form.

---

## 2. Decoder LSTM: Returning Full Sequences and States

The decoder’s LSTM must output a **prediction at every timestep**, so we set `return_sequences=True`. We also keep `return_state=True`, because the final hidden and cell states will be useful later (especially during inference):

```python
decoder_lstm = LSTM(100, return_sequences=True, return_state=True)
# This time we care about full return sequences
```

* `100` → dimensionality of the LSTM’s hidden state (you previously used 256; 100 is just the example in the text).
* `return_sequences=True` → we get an output vector for **each timestep**, not just the last one.
* `return_state=True` → we still obtain the final hidden and cell states (`decoder_state_hidden`, `decoder_state_cell`).

---

## 3. Linking Encoder and Decoder via Initial State

The critical difference from the encoder is how the decoder is **initialized**.
Instead of starting from zero states, the decoder LSTM is initialized with the **final states of the encoder**, which carry the compressed meaning of the input sentence:

```python
decoder_outputs, decoder_state_hidden, decoder_state_cell = decoder_lstm(
    decoder_inputs,
    initial_state=encoder_states
)
```

Where:

* `decoder_inputs` → one-hot Spanish tokens (shifted sequence with `<START>`).
* `encoder_states` → `[state_hidden, state_cell]` from the encoder LSTM.
* `decoder_outputs` → output at each timestep (sequence of hidden vectors).
* `decoder_state_hidden`, `decoder_state_cell` → final states of the decoder (useful for step-by-step decoding during inference).

During **training**, we primarily care about `decoder_outputs`, because each timestep of this sequence will be mapped to a probability distribution over all possible target tokens.

---

## 4. Dense + Softmax: From Hidden States to Token Probabilities

The LSTM’s hidden state at each timestep has size equal to the LSTM dimensionality (e.g., 100), but we need a vector of size `num_decoder_tokens` that represents **probabilities over all target tokens**. That’s the job of the final Dense layer:

```python
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)
```

* `Dense(num_decoder_tokens, ...)` maps each LSTM output from shape `[batch_size, timesteps, 100]` to `[batch_size, timesteps, num_decoder_tokens]`.
* `activation='softmax'` ensures that for each timestep:

  * all values are between 0 and 1
  * the probabilities across all tokens sum to 1

So at every timestep, the decoder now outputs a **probability distribution over the entire target vocabulary** (e.g., “0.7 for ‘Hola’, 0.2 for ‘Adiós’, 0.1 split among others”).

---

## 5. Why a Dense Layer (and Why Softmax)?

Keras could use different kinds of output layers, but `Dense` is the simplest and most common for classification-like tasks at each timestep. Here the “classes” are the **target tokens**:

* The LSTM handles **temporal dependencies** (what came before).
* The Dense + Softmax layer turns each hidden vector into a **token prediction**.

Together with teacher forcing and the encoder states, this setup allows the decoder to:

1. Start from the encoder’s understanding of the input sentence.
2. Read the previous target token (from `decoder_inputs`).
3. Output a probability distribution over the next token (via `decoder_outputs` after the Dense+Softmax layer).

This is how the model learns to generate full translations, one token at a time.


In [18]:
from tensorflow import keras
# Step 1: Import Input, LSTM, and Dense
from keras.layers import Input, LSTM, Dense
from keras.models import Model

# ---------------------------------------------------------
# ENCODER SETUP
# ---------------------------------------------------------
# Step 2: Create encoder input layer
encoder_inputs = Input(shape=(None, num_encoder_tokens))

# Step 3: Encoder LSTM (returns final hidden & cell states)
encoder_lstm = LSTM(256, return_state=True)

# Step 4: Extract encoder outputs and states
encoder_outputs, state_hidden, state_cell = encoder_lstm(encoder_inputs)

# Step 5: Package encoder states to pass into decoder
encoder_states = [state_hidden, state_cell]

# ---------------------------------------------------------
# DECODER SETUP
# ---------------------------------------------------------
# Step 6: Decoder input layer (target-language one-hot vectors)
decoder_inputs = Input(shape=(None, num_decoder_tokens))

# Step 7: Decoder LSTM (returns full sequences + final states)
decoder_lstm = LSTM(256, return_sequences=True, return_state=True)

# Step 8: Run decoder LSTM using encoder states as initial state
decoder_outputs, decoder_state_hidden, decoder_state_cell = decoder_lstm(
    decoder_inputs,
    initial_state=encoder_states
)

# ---------------------------------------------------------
# FINAL DENSE + SOFTMAX LAYER
# ---------------------------------------------------------
# Step 9: Build a Dense layer that outputs probabilities over all decoder tokens
decoder_dense = Dense(num_decoder_tokens, activation="softmax")

# Step 10: Apply the Dense layer to decoder outputs
decoder_outputs = decoder_dense(decoder_outputs)


# Build and Train the Seq2Seq Model

Once the encoder and decoder are wired together, we wrap them into a single **sequence-to-sequence (seq2seq)** model, compile it, and train it on our one-hot encoded data.

---

## 1. Defining the Seq2Seq Model

Our model has:

* **Two inputs:**

  * `encoder_inputs` → one-hot encoded source sentences (e.g., English)
  * `decoder_inputs` → one-hot encoded shifted target sentences (e.g., Spanish with `<START>`)
* **One output:**

  * `decoder_outputs` → probability distributions over the target vocabulary at each timestep

We connect everything via Keras’ `Model` API:

```python
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
```

This tells Keras:

> “Given both the encoder input sequence and the decoder input sequence, learn to produce the decoder output sequence.”

---

## 2. Compiling the Model

Before training, we must **compile** the model by specifying:

* an **optimizer** → how weights are updated
* a **loss function** → how wrong each prediction is
* optional **metrics** → extra statistics to monitor (e.g. accuracy)

In this setup:

* **Optimizer:** `"rmsprop"`
  A variant of gradient descent that adapts the learning rate per parameter and often works well for recurrent models.

* **Loss:** `"categorical_crossentropy"`
  The standard loss for multi-class classification when targets are one-hot vectors. At each timestep, the model is choosing among all possible target tokens.

* **Metric:** `"accuracy"`
  Tracks how often the predicted token with highest probability matches the true token.

```python
model.compile(
    optimizer='rmsprop',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
```

---

## 3. Fitting (Training) the Model

To train the model, we call `.fit()` with:

* **Inputs:**

  * `[encoder_input_data, decoder_input_data]`

    * `encoder_input_data` → English sentences as 3D tensors of one-hot vectors
      shape: `(num_samples, max_encoder_seq_length, num_encoder_tokens)`
    * `decoder_input_data` → Spanish sentences shifted with `<START>` at the beginning
      shape: `(num_samples, max_decoder_seq_length, num_decoder_tokens)`

* **Targets:**

  * `decoder_target_data` → Spanish sentences shifted one step ahead
    (what the decoder should output at each timestep)

* **Hyperparameters:**

  * `batch_size=10` → number of sentence pairs processed before each weight update
  * `epochs=100` → how many passes over the entire dataset
  * `validation_split=0.2` → 20% of the data is kept aside for validation

```python
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=10,
    epochs=100,
    validation_split=0.2
)
```

Conceptually, at each training step the model sees:

* an **English input sequence** (to encode),
* a **Spanish input sequence** with `<START>` tokens (to drive the decoder),
* and it learns to produce the **next Spanish tokens** in `decoder_target_data` using **teacher forcing**.

---

## 4. What We Get After Training

After training:

* The encoder has learned how to compress an input sentence into hidden states.
* The decoder has learned how to use those states, plus previous target tokens, to predict the next token at each timestep.
* Together, they form a seq2seq model that can be adapted for translation or other sequence generation tasks (with additional inference-time wiring).


In [20]:
print(encoder_inputs)
print(decoder_inputs)
print(decoder_outputs)

<KerasTensor shape=(None, None, 18), dtype=float32, sparse=False, name=keras_tensor_4>
<KerasTensor shape=(None, None, 27), dtype=float32, sparse=False, name=keras_tensor_8>
<KerasTensor shape=(None, None, 27), dtype=float32, sparse=False, name=keras_tensor_12>


In [19]:
# ---------------------------------------------------------
# BUILD TRAINING MODEL
# ---------------------------------------------------------
training_model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

print("Model summary:\n")
training_model.summary()
print("\n\n")

# ---------------------------------------------------------
# COMPILE THE MODEL
# ---------------------------------------------------------
training_model.compile(
    optimizer="rmsprop",
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)

# ---------------------------------------------------------
# TRAINING CONFIGURATION
# ---------------------------------------------------------
batch_size = 50
epochs = 50

print("Training the model:\n")

# ---------------------------------------------------------
# FIT THE MODEL
# ---------------------------------------------------------
training_model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
)

Model summary:



Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, None, 18)  │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_2       │ (None, None, 27)  │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ [(None, 256),     │    281,600 │ input_layer_1[0]… │
│                     │ (None, 256),      │            │                   │
│                     │ (None, 256)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_2 (LSTM)       │ [(None, None,     │    290,816 │ input_layer_2[0]… │
│                     │ 256), (None,      │            │ lstm_1[0][1],     │
│                     │ 256), (None,      │            │ lstm_1[0][2]      │
│                     │ 256)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, None, 27)  │      6,939 │ lstm_2[0][0]      │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 579,355 (2.21 MB)

 Trainable params: 579,355 (2.21 MB)

 Non-trainable params: 0 (0.00 B)




Training the model:

Epoch 1/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - accuracy: 0.0625 - loss: 1.3057 - val_accuracy: 0.0000e+00 - val_loss: 1.4596
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 197ms/step - accuracy: 0.0625 - loss: 1.2986 - val_accuracy: 0.1389 - val_loss: 1.4554
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 169ms/step - accuracy: 0.1146 - loss: 1.2926 - val_accuracy: 0.1389 - val_loss: 1.4514
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 173ms/step - accuracy: 0.1146 - loss: 1.2869 - val_accuracy: 0.1389 - val_loss: 1.4473
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 174ms/step - accuracy: 0.1042 - loss: 1.2810 - val_accuracy: 0.1111 - val_loss: 1.4427
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 163ms/step - accuracy: 0.0833 - loss: 1.2747 - val_accuracy: 0.0833 - val_loss: 1.4372
Epoch 7/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 154ms/step - accuracy: 0.0833 - loss: 1.2672 - val_accuracy: 0.0833 - val_loss: 1.4298
Epoch 8/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 158ms/step - accuracy: 0.0833 - loss: 1.2577 - val_acc

# Setup for Testing (Seq2Seq Inference)

After training, we want to **generate translations** for new English sentences. During training we used **teacher forcing** (we knew the whole Spanish sentence and fed it in at once), but at test time we **don’t know the target sequence**. We must decode **step by step**, predicting one token at a time and feeding predictions back into the model.

To do that, we split the trained seq2seq network into two separate models:

* an **encoder model** (turns an input sentence into final states)
* a **decoder model** (uses those states to generate the target sentence one token at a time)

---

## 1. Encoder Model for Inference

The encoder’s job at inference time is simple:

* Take a new English sentence as input.
* Return the final hidden and cell states of the encoder LSTM.

We reuse the trained `encoder_inputs` and `encoder_states` to define an **inference encoder model**:

```python
encoder_model = Model(encoder_inputs, encoder_states)
```

* Input: `encoder_inputs` (one-hot or embedded English sequence).
* Output: `encoder_states = [state_hidden, state_cell]` from the trained encoder LSTM.

These states summarize the entire input sentence and will be used to initialize the decoder.

---

## 2. Placeholder Inputs for Decoder States

At test time, decoding is iterative:

1. Start with the encoder’s final states.
2. Predict the next token.
3. Use the **updated** decoder states to predict the next token again.
4. Repeat until `<END>` is produced or a max length is reached.

To support this loop, we create **input layers** that act as placeholders for the decoder’s hidden and cell states:

```python
latent_dim = 256  # must match the LSTM dimensionality

decoder_state_input_hidden = Input(shape=(latent_dim,))
decoder_state_input_cell = Input(shape=(latent_dim,))

decoder_states_inputs = [decoder_state_input_hidden, decoder_state_input_cell]
```

* `decoder_state_input_hidden` → placeholder for previous hidden state `h_{t-1}`
* `decoder_state_input_cell` → placeholder for previous cell state `c_{t-1}`
* `decoder_states_inputs` → list grouping both together

On the **first decoding step**, these will be filled with the encoder’s final states. On subsequent steps, they’ll be filled with the decoder’s own updated states from the previous timestep.

---

## 3. Running the Decoder LSTM One Step at a Time

We now reuse the **trained** decoder components:

* `decoder_lstm` → the LSTM layer we trained for the decoder
* `decoder_dense` → the Dense + softmax layer that maps hidden states to token probabilities

At inference, the decoder LSTM is called with:

* `decoder_inputs` → the current token’s one-hot vector (e.g. `<START>` or previous prediction)
* `decoder_states_inputs` → the previous timestep’s hidden and cell states

```python
decoder_outputs, state_hidden, state_cell = decoder_lstm(
    decoder_inputs,
    initial_state=decoder_states_inputs
)

# New states after processing one timestep:
decoder_states = [state_hidden, state_cell]

# Map LSTM outputs to probability distribution over target tokens:
decoder_outputs = decoder_dense(decoder_outputs)
```

* `decoder_outputs` → probabilities over all Spanish tokens for the current timestep.
* `decoder_states` → updated hidden and cell states to be fed back in the **next** decoding step.

---

## 4. Building the Decoder Inference Model

Finally, we wrap this logic into a **decoder model** that, given:

* the current token (`decoder_inputs`)
* the previous hidden/cell states (`decoder_states_inputs`)

returns:

* the token probabilities for the current step (`decoder_outputs`)
* the updated hidden/cell states (`decoder_states`)

```python
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states
)
```

So during inference, the decoding loop looks like:

1. Use `encoder_model` to get encoder states from an English sentence.
2. Initialize `decoder_model` with:

   * input token = `<START>`
   * states = encoder states
3. Get:

   * predicted next-token distribution
   * updated states
4. Choose next token (e.g., argmax), feed it back as `decoder_inputs` together with updated states.
5. Repeat until `<END>` or max length.

This step-by-step setup makes the trained seq2seq network capable of **generating translations autonomously**, without knowing the target sequence in advance.


In [21]:
# ---------------------------------------------------------
# Step 2: Extract encoder layers and build encoder_model
# ---------------------------------------------------------
# training_model.input[0] = encoder input layer
encoder_inputs = training_model.input[0]

# training_model.layers[2].output contains:
# (encoder_outputs, state_h_enc, state_c_enc)
encoder_outputs, state_h_enc, state_c_enc = training_model.layers[2].output

# Final encoder states used to initialize decoder at inference
encoder_states = [state_h_enc, state_c_enc]

# Encoder inference model
encoder_model = Model(encoder_inputs, encoder_states)

# ---------------------------------------------------------
# Step 3: Build decoder state input placeholders for inference
# ---------------------------------------------------------
latent_dim = 256  # must match the training LSTM size

decoder_state_input_hidden = Input(shape=(latent_dim,))
decoder_state_input_cell = Input(shape=(latent_dim,))

# Combine into list
decoder_states_inputs = [decoder_state_input_hidden, decoder_state_input_cell]

# ---------------------------------------------------------
# Step 4: Run trained decoder LSTM for one timestep at inference
# ---------------------------------------------------------
decoder_outputs, state_hidden, state_cell = decoder_lstm(
    decoder_inputs,
    initial_state=decoder_states_inputs
)

# Save new states
decoder_states = [state_hidden, state_cell]

# ---------------------------------------------------------
# Step 5: Pass outputs through the trained Dense layer
# ---------------------------------------------------------
decoder_outputs = decoder_dense(decoder_outputs)

# ---------------------------------------------------------
# Step 6: Build final decoder inference model
# ---------------------------------------------------------
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states
)

# The Test Function for Seq2Seq Inference

Once the encoder–decoder models are trained and we’ve built separate **inference models**, we can finally test the system on a new English sentence. For that, we create a **test (decode) function** that:

1. Takes a **NumPy matrix** representing the English input sentence.
2. Uses the **encoder model** to get the sentence representation (states).
3. Uses the **decoder model** step by step to generate the Spanish translation.

---

## 1. Getting the Encoder States from the Test Input

At test time, the English sentence is already preprocessed and vectorized into a NumPy array with shape:

* `(1, max_encoder_seq_length, num_encoder_tokens)`

We pass this test input through the **encoder inference model**:

```python
# test_input is a NumPy matrix representing one English sentence
states = encoder_model.predict(test_input)
```

* `encoder_model` returns the **final hidden state** and **cell state** of the encoder LSTM.
* These states summarize the meaning of the entire English sentence.
* We will use these as the **initial state** for the decoder.

---

## 2. Initializing the Target Sequence for the Decoder

The decoder works token by token. At the very start of translation, we need an initial **target sequence** of shape:

* batch size = `1` (we decode one sentence at a time)
* timesteps = `1` (we start with just one token)
* features = `num_decoder_tokens` (size of the Spanish vocabulary)

So we create:

```python
target_sequence = np.zeros((1, 1, num_decoder_tokens))
```

This is a **3D one-hot tensor** where we will set exactly one position to 1 for the first timestep.

---

## 3. Seeding the Decoder with the `<START>` Token

We know that every generated Spanish sentence must begin with a **start token**, e.g. `"<START>"`. We one-hot encode this token in our `target_sequence`:

```python
target_sequence[0, 0, target_features_dict['<START>']] = 1.
```

* `target_features_dict` maps each Spanish token (like `"<START>"`, `"Estoy"`, `"feliz"`, `"<END>"`, etc.) to an index.
* We set the entry for `"<START>"` to `1.` and leave all others at `0.`

This tells the decoder:

> “At timestep 0, the previous token is `<START>` — now predict the next word.”

---

## 4. Preparing the String for the Generated Translation

As the decoder predicts tokens step by step, we need somewhere to accumulate them as a human-readable sentence. We start with an empty string:

```python
decoded_sentence = ''
```

Inside the decoding loop, we’ll:

1. Run the decoder on `target_sequence` + current states.
2. Get a probability distribution over the next token.
3. Pick the **most likely token** (e.g., using `argmax`).
4. Append that token to `decoded_sentence`.
5. Update `target_sequence` to contain this new token.
6. Feed the updated `target_sequence` and updated states back into the decoder.

We continue until we generate the `"<END>"` token or reach a maximum length.
Finally, the function returns `decoded_sentence` as the Spanish translation of the input English sentence.


In [24]:
# ---------------------------------------------------------
# Decode function required by the instructions
# ---------------------------------------------------------
def decode_sequence(test_input):
    # Step 1: Encode input sentence → get encoder final states
    encoder_states_value = encoder_model.predict(test_input)

    # Step 2: Initialize decoder states with encoder states
    decoder_states_value = encoder_states_value

    # Step 3: Create an empty target sequence (batch 1, timestep 1)
    target_seq = np.zeros((1, 1, num_decoder_tokens))

    # Step 4: Set the <START> token to 1
    target_seq[0, 0, target_features_dict["<START>"]] = 1.

    decoded_sentence = ''
    return decoded_sentence

# ---------------------------------------------------------
# Test the decode function on first 10 samples
# ---------------------------------------------------------
for seq_index in range(10):
    test_input = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(test_input)
    print('-')
    print('Input sentence:', input_docs[seq_index])
    print('Decoded sentence:', decoded_sentence)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step
-
Input sentence: We'll see.
Decoded sentence: 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
-
Input sentence: We'll see.
Decoded sentence: 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step
-
Input sentence: We'll try.
Decoded sentence: 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
-
Input sentence: We've won!
Decoded sentence: 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
-
Input sentence: Well done.
Decoded sentence: 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
-
Input sentence: What's up?
Decoded sentence: 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
-
Input sentence: Who cares?
Decoded sentence: 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
-
Input sentence: Who drove?
Decoded sentence: 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
-
Input sentence: Who drove?
Decoded sentence: 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
-
Input sentence: Who is he?
Decoded sentence: 


# Test Function (Part 2): Generating Text with the Trained Seq2Seq Model

Now that we have separate **encoder** and **decoder** models for inference, we can finally use them to *generate* a Spanish translation from an English sentence. This happens inside the `decode_sequence()` function and is done **one token at a time**.

The key idea:

* The **encoder** compresses the entire English sentence into two state vectors.
* The **decoder** takes those states and **generates the Spanish sentence step by step**, updating its internal state after each predicted word.

---

## 1. When Do We Stop Decoding?

We don’t want the model to generate infinite text, so we decode in a `while` loop and stop when:

1. The model generates the token `"<END>"`, **or**
2. The length of the decoded Spanish sentence reaches `max_decoder_seq_length`.

The skeleton looks like:

```python
decoded_sentence = ""

while True:
    # 1. Get predictions from decoder_model
    # 2. Choose the most probable token
    # 3. Append token to decoded_sentence
    # 4. Stop if <END> or max length
    # 5. Prepare next input token and states
    pass
```

---

## 2. Running One Decoder Step

At each step, we call the **decoder inference model** with:

* `target_seq` → the current input token to the decoder (initially `<START>`)
* `decoder_states_value` → the current hidden and cell states

```python
output_tokens, new_decoder_hidden_state, new_decoder_cell_state = decoder_model.predict(
    [target_seq] + decoder_states_value
)
```

Here:

* `output_tokens` is a 3D tensor with shape `(1, 1, num_decoder_tokens)`

  * It contains the **probability distribution** over all Spanish tokens at the current timestep.
* `new_decoder_hidden_state` and `new_decoder_cell_state` are the **updated states** after processing this token. They will be used for the *next* decoding step.

---

## 3. Choosing the Next Token

We take the most probable token (greedy decoding) using `np.argmax()` on the last timestep of `output_tokens`:

```python
sampled_token_index = np.argmax(output_tokens[0, -1, :])
sampled_token = reverse_target_features_dict[sampled_token_index]

decoded_sentence += " " + sampled_token
```

* `output_tokens[0, -1, :]` selects:

  * batch `0`
  * last timestep (`-1`)
  * all token probabilities
* `np.argmax(...)` gives us the index of the highest-probability token.
* `reverse_target_features_dict` maps indices back to actual Spanish words/tokens.
* We append this token to `decoded_sentence`.

---

## 4. Stopping Condition: `<END>` or Max Length

Inside the loop, after we pick `sampled_token`, we should check if we’re done:

* If `sampled_token == "<END>"`, we stop (and usually don’t include `<END>` in the final printed translation).
* If `len(decoded_sentence.split())` reaches `max_decoder_seq_length`, we also stop to avoid runaway generation.

Conceptually:

```python
if (sampled_token == "<END>") or (len(decoded_sentence.split()) > max_decoder_seq_length):
    break
```

---

## 5. Preparing for the Next Step

If we’re not done yet, we must:

1. Build the next `target_seq` containing the **newly sampled token** in one-hot form.
2. Update `decoder_states_value` with the **new states** from this step so they carry context forward.

```python
# 1. Build the next input token for the decoder
target_seq = np.zeros((1, 1, num_decoder_tokens))
target_seq[0, 0, sampled_token_index] = 1.

# 2. Update states for the next iteration
decoder_states_value = [new_decoder_hidden_state, new_decoder_cell_state]
```

On the **next iteration** of the `while` loop:

* `target_seq` contains the one-hot encoding of the **previously predicted word**.
* `decoder_states_value` carries the decoder’s updated “memory” so it can condition the next prediction on the entire partial sentence generated so far.

---

## 6. Putting It All Together

Inside `decode_sequence(test_input)`:

1. Use `encoder_model.predict(test_input)` to get the initial decoder states.
2. Initialize `target_seq` with the `<START>` token.
3. Initialize `decoded_sentence = ''`.
4. Run a `while` loop:

   * Call `decoder_model.predict(...)` with `[target_seq] + decoder_states_value`.
   * Choose the next token using `argmax`.
   * Append token to `decoded_sentence`.
   * Break if `<END>` or max length reached.
   * Update `target_seq` and `decoder_states_value`.
5. Return `decoded_sentence` (often with `<END>` removed before printing).

This loop is how the trained seq2seq model **generates text**: one token at a time, always using its own previous predictions and internal states as context, just like a small, hand-built translation engine.


In [25]:
# -----------------------------
# Sequence Decoding Function
# -----------------------------
def decode_sequence(test_input):
    # Encode input sentence → initial decoder states
    encoder_states_value = encoder_model.predict(test_input)
    decoder_states_value = encoder_states_value

    # Prepare starting token sequence: <START>
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_features_dict["<START>"]] = 1.

    decoded_sentence = ""
    stop_condition = False

    while not stop_condition:
        # Predict next token probabilities + states
        output_tokens, new_h, new_c = decoder_model.predict(
            [target_seq] + decoder_states_value
        )

        # Pick most probable token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_token = reverse_target_features_dict[sampled_token_index]

        decoded_sentence += " " + sampled_token

        # Stop if <END> or too long
        if sampled_token == "<END>" or len(decoded_sentence.split()) > max_decoder_seq_length:
            stop_condition = True
        else:
            # Update the target sequence with the new predicted token
            target_seq = np.zeros((1, 1, num_decoder_tokens))
            target_seq[0, 0, sampled_token_index] = 1.

            # Update decoder states
            decoder_states_value = [new_h, new_c]

    return decoded_sentence

# -----------------------------
# Try decoding the first 10 sequences
# -----------------------------
for seq_index in range(10):
    test_input = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(test_input)
    print("-")
    print("Input sentence:", input_docs[seq_index])
    print("Decoded sentence:", decoded_sentence)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 603ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
-
Input sentence: We'll see.
Decoded sentence:  . . . <END>
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
-
Input sentence: We'll see.
Decoded sentence:  . . . <END>
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
-
Input sentence: We'll try.
Decoded sentence:  . . . <END>
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
-
Input sentence: We've won!
Decoded sentence:  ¿ . . <END